In [ ]:
using Plots
using Parameters
using DifferentialEquations

In [ ]:
using LabelledArrays

# Beeler-Reuter model

[Beeler Reuter model on CellML](https://models.physiomeproject.org/e/23a/beeler_reuter_1977.cellml/view)

## Cell scheme
![Image of Beeler-Reuter model](https://models.physiomeproject.org/w/miller/beeler_reuter_1977_uncertexample/@@rawfile/b5412f13ad89db3cecd00b6c68c1d6e3308e2110/beeler_reuter_1977.png)

## Diagram of the current flows across the cell membrane

![Image of Beeler-Reuter scheme](https://models.physiomeproject.org/w/miller/beeler_reuter_1977_uncertexample/@@rawfile/b5412f13ad89db3cecd00b6c68c1d6e3308e2110/cellml_rendering.gif)

# Gating variables

In [ ]:
function calculate_alpha_m(V)
    alpha_m = -1. * (V + 47.0) / (exp(-0.1 * (V+47.)) - 1)
end

function calculate_beta_m(V)
    beta_m = 40. * exp(-0.056 * (V + 72.))
end

function calculate_alpha_h(V)
    alpha_h = 0.126 * exp(-0.25 * (V + 77.))
end

function calculate_beta_h(V)
    beta_h = 1.7/(1 + exp(-0.082 * (V + 22.5)))
end

function calculate_alpha_j(V)
    alpha_j = 0.055 * exp(-0.25 * (V + 78.)) / (1. + exp(-0.2 * (V + 78.)))
end

function calculate_beta_j(V)
    beta_j = 0.3 / (1. + exp(-0.1 * (V + 32.)))
end

function calculate_alpha_d(V)
    alpha_d = 0.095 * exp(-(V - 5.)/100.) / (1. + exp(-(V - 5.) / 13.89))
end

function calculate_beta_d(V)
    beta_d = 0.07 * exp(-(V + 44.) / 59.) / (1. + exp((V + 44.) / 20.))
end

function calculate_alpha_f(V)
    alpha_f = 0.012 * exp(-(V + 28.) / 125.) / (1. + exp((V + 28.) / 6.67))
end

function calculate_beta_f(V)
    beta_f = 0.0065 * exp(-(V + 30.) / 50.) / ( 1. + exp(-(V + 30.) / 5.))
end

function calculate_alpha_x1(V)
    alpha_x1 = 5e-4 * exp((V + 50.) / 12.1) / (1. + exp((V + 50.) / 17.5))
end

function calculate_beta_x1(V)
    beta_x1 = 0.0013 * exp(-(V + 20.) / 16.67) / (1. + exp(-(V + 20.) / 25.) )
end

# Algebraic ~ observables

In [ ]:
function calculate_E_s!(u, p, t, a)
    @unpack Cai = u
    E_s = -82.3 - (13.0287 * log(Cai * 0.001))
    @pack! a = E_s
end

function calculate_i_s!(u, p, t, a)
    @unpack V, d, f = u
    @unpack g_s = p
    @unpack E_s = a
    i_s = g_s * d * f * (V - E_s)
    @pack! a = i_s
end

function calculate_i_Na!(u, p, t, a)
    @unpack V, m, h, j = u
    @unpack g_Na, g_Nac, E_Na = p
    i_Na = (g_Na * m^3 * h * j + g_Nac) * (V - E_Na)
    @pack! a = i_Na
end

function calculate_i_x1!(u, p, t, a)
    @unpack V, x1 = u
    i_x1 = x1 * 8e-3 * (exp(0.004 * (V + 77.)) - 1.) / (exp(0.04 * (V + 35.)))
    @pack! a = i_x1
end

function calculate_i_K1!(u, p, t, a)
    @unpack V = u
    @unpack E_K = p
    
    i_K1 = 0.0035 * (4. * (exp(0.04 * (V+E_K)) - 1) / (exp(0.08 * (V + 53.)) + exp(0.04 * (V + 53.)))
                      + 0.2 * (V + 23.) / (1 - exp(-0.04 * (V + 23.)))) 
    @pack! a = i_K1
end

function calculate_Istim!(u, p, t, a)
    @unpack IstimStart, IstimEnd, IstimPeriod, IstimPulseDuration, IstimAmplitude = p 
    condition = (IstimStart <= t <= IstimEnd && (t - IstimStart) - floor((t - IstimStart) / IstimPeriod) * IstimPeriod <= IstimPulseDuration)
    Istim = condition ? IstimAmplitude : 0.0
    @pack! a = Istim
end

In [ ]:
function calculate_algebraic!(u, p, t, a)

    @unpack V = u
    
    a.alpha_m = calculate_alpha_m(V)
    a.beta_m = calculate_beta_m(V)
    
    a.alpha_h = calculate_alpha_h(V)
    a.beta_h = calculate_beta_h(V)
    
    a.alpha_j = calculate_alpha_j(V)
    a.beta_j = calculate_beta_j(V)
    
    a.alpha_d = calculate_alpha_d(V)
    a.beta_d = calculate_beta_d(V)
    
    a.alpha_f = calculate_alpha_f(V)
    a.beta_f = calculate_beta_f(V)
    
    a.alpha_x1 = calculate_alpha_x1(V)
    a.beta_x1 = calculate_beta_x1(V)
    
    calculate_E_s!(u, p, t, a)
    calculate_i_s!(u, p, t, a)
    calculate_i_Na!(u, p, t, a)
    calculate_i_x1!(u, p, t, a)
    calculate_i_K1!(u, p, t, a)
    calculate_Istim!(u, p, t, a)
    
end

In [ ]:
function calculate_d_gate(gate, alpha, beta, scaler = 1)
    d_gate = scaler * (alpha * (1. - gate) - beta * gate)
end


function calculate_d_Cai(du, u, p, t, a)
    @unpack Cai = u
    @unpack i_s = a
    d_Cai = -0.01 * i_s + 0.07 * (0.0001 - Cai)
end


function calculate_d_V(du, u, p, t, a)
    @unpack C = p
    @unpack Istim, i_Na, i_s, i_x1, i_K1 = a
    d_V = (Istim - (i_Na + i_s + i_x1 + i_K1)) / C
end

In [ ]:
function calculate_rates!(du, u, p, t, a)

    @unpack V, m, h, j, Cai, d, f, x1 = u
    calculate_algebraic!(u, p, t, a)
    @unpack m_scaler = p
    du.m = calculate_d_gate(m, a.alpha_m, a.beta_m, m_scaler)
    du.h = calculate_d_gate(h, a.alpha_h, a.beta_h)
    du.j = calculate_d_gate(j, a.alpha_j, a.beta_j)
    du.d = calculate_d_gate(d, a.alpha_d, a.beta_d)
    du.f = calculate_d_gate(f, a.alpha_f, a.beta_f)
    du.x1 = calculate_d_gate(x1, a.alpha_x1, a.beta_x1)

    du.V = calculate_d_V(du, u, p, t, a)
    du.Cai = calculate_d_Cai(du, u, p, t, a)

end

# Define problem

In [ ]:
u₀ = LVector((V = -85.0,
              m = 0.011,
              h = 0.998,
              j = 0.975,
              Cai = 1e-4,
              d = 0.003,
              f = 0.994,
              x1 = 0.0001))

In [ ]:
p_initial = LVector((
    C = 0.01,
    g_Na = 4e-2,
    E_Na = 50,
    E_K = 85,
    g_Nac = 3e-5,
    g_s = 9e-4,
    m_scaler = 1.,
    IstimStart = 10,
    IstimEnd = 50000,
    IstimAmplitude = 0.5,
    IstimPeriod = 1000,
    IstimPulseDuration = 1))

In [ ]:
tspan = (0., 1500.)

In [ ]:
a_syms = (:alpha_m, :beta_m,
          :alpha_h, :beta_h,
          :alpha_j, :beta_j,
          :alpha_d, :beta_d,
          :alpha_f, :beta_f,
          :alpha_x1, :beta_x1,
          :E_s, :i_s, :i_Na, :i_x1, :i_K1, :Istim)

a = @LVector Real a_syms
calculate_algebraic!(u₀, p_initial, 0., a)  # let's check
a

In [ ]:
du = similar(u₀)
u = similar(u₀)
calculate_rates!(du, u, p_initial, 0., a)  # let's check

In [ ]:
rhs = ODEFunction((du, u, p, t) -> calculate_rates!(du, u, p, t, a))
prob = ODEProblem(rhs, u₀, tspan, p_initial)

# Solve

In [ ]:
sol = solve(prob, Rodas5(), dt=1e-3, dtmax=0.1);

In [ ]:
plot(sol, vars=[:V], ylabel = "V, mV")


# Restore algebraics

In [ ]:
algebraics = Vector{typeof(a)}()

for (t, u) in zip(sol.t, sol.u)
    calculate_algebraic!(u, p_initial, t, a)
    append!(algebraics, [deepcopy(a)])
end

In [ ]:
a_matrix = hcat(algebraics...)
algebraics = (;zip(a_syms, eachrow(a_matrix))...);

In [ ]:
plot(algebraics.i_Na, label = "i_Na")

# Hyperkalemia

In [ ]:
function calculate_i_K1(V, V_rev)
    i_K1 = 0.0035 * (4. * (exp(0.04 * (V+V_rev)) - 1) / (exp(0.08 * (V + 53.)) + exp(0.04 * (V + 53.)))
                      + 0.2 * (V + 23.) / (1 - exp(-0.04 * (V + 23.)))) 
end


In [ ]:
p = deepcopy(p_initial)
v_rev = 85:-3:65
v_range = -100:-30
E_K_critical = 75
tspan = (0., 3000.)

In [ ]:
hline([0.0], linecolor = :black, label = false)
for v_ in v_rev
    i_K1 = []
    for v in v_range
        push!(i_K1, calculate_i_K1(v, v_))
    end
    condition = (v_ < E_K_critical)
    plot!(v_range, i_K1, label = "v_rev = " * string(v_) * " mV", linecolor = (condition ? :blue : :red))
end
plot!(xlims = (-100, -20), ylims = (-0.1,0.1), xlabel = "V, mV", ylabel = "I_K1 ")

In [ ]:
plot()
for v_ in v_rev
    p[:E_K] = v_
    prob = ODEProblem(rhs, u₀, tspan, p)
    sol = solve(prob, Rodas5(), dt=1e-3, dtmax=0.1);
    condition = (v_ < E_K_critical)
    plot!(sol, vars=[:V], linecolor = (condition ? :blue : :red), label = "v_rev = " * string(v_) * " mV")
end
plot!(ylabel = "V, mV")

# Activation characteristics

In [ ]:
tau_m = []
tau_h = []
tau_j = []
m = []
h = []
j = []
v_range = -100:1.1:-10
for v in v_range
    push!(tau_m, 1/(calculate_alpha_m(v) + calculate_beta_m(v)))
    push!(tau_h, 1/(calculate_alpha_h(v) + calculate_beta_h(v)))
    push!(tau_j, 1/(calculate_alpha_j(v) + calculate_beta_j(v)))
    push!(m, calculate_alpha_m(v)/(calculate_alpha_m(v) + calculate_beta_m(v)))
    push!(h, calculate_alpha_h(v)/(calculate_alpha_h(v) + calculate_beta_h(v)))
    push!(j, calculate_alpha_j(v)/(calculate_alpha_j(v) + calculate_beta_j(v)))


end

In [ ]:
plot(v_range, [tau_m , tau_h], yaxis= :log, label = ["tau_m" "tau_h"], lw = 2, ylabel = "t, ms", xlabel = "V, mV")

In [ ]:
plot(v_range, [m, h], label = ["m" "h"], lw = 2, xlabel = "V, mV")


In [ ]:
tspan = (0., 600.)
p = deepcopy(p_initial)
coeff_critical = maximum(tau_m)/maximum(tau_h)
coeff_critical_log = log10(coeff_critical) 

In [ ]:
coeff = -2.:0.5:1. 
sol_arrays = []
for c in coeff
    p[:m_scaler] = 10^c
    prob = ODEProblem(rhs, u₀, tspan, p)
    sol = solve(prob, Rodas5(), dt=1e-3, dtmax=0.1)
    push!(sol_arrays, sol)
end


In [ ]:
plot()
for k in 1:length(sol_arrays)
    condition = (coeff[k] < 0.)
    plot!(sol_arrays[k], vars=[:V], lw = 2,label = "τₘ/τₕ ~ 10^" *string(round(- coeff[k] + coeff_critical_log, digits=1)) ,  linecolor = (condition ? :blue : :red),)
end
plot!(ylabel = "V, mV")

In [ ]:
I_Na_max = []
for sol in sol_arrays
    algebraics = Vector{typeof(a)}()
    for (t, u) in zip(sol.t, sol.u)
        calculate_algebraic!(u, p_initial, t, a)
        append!(algebraics, [deepcopy(a)])
    end
    a_matrix = hcat(algebraics...)
    algebraics = (;zip(a_syms, eachrow(a_matrix))...);
    push!(I_Na_max, minimum(algebraics.i_Na))
end


In [ ]:
plot(10 .^(- coeff .+ coeff_critical_log), I_Na_max, m = 5,  xaxis= :log, label = false, xlabel = "τₘ/τₕ", ylabel = "Amplitude of I Na, nA", lw = 2)